# MyHDL Legacy code migration notes

When migrating legacy code to myHDL.v2we, the thumb rules are:

* **Hierarchical resolution and (optional) strict typing in place**
* **Everything outside a hardware generator (see below) is a reference**:
  * IRL constructs allow references to logical combinations
  * Everything instanced by a variable in the `@block` context is an instance, signal, or reference and may be resolved into
    hardware
* Clean up boolean constructs (see [Boolean Logic operations](../notebooks/myhdl_changes.ipynb#Logic-operations)):
  * Make sure to use boolean logic only within conditional statements
  * Translate to binary logic operations when assigning to a signal (**MIND THE OPERATOR PRECEDENCE!**)
* Clean up variable usage (see [Variable usage](#Variable-usage)):
  * First occurence of a variable (within a generator context only) defines its data type
  * Do not use variables where possible, rather reserve an auxiliary signal or use a reference (outside the generator)
* Function interface:
  * Use strict typing for port types and generics that should resolve to HDL
  * `@block`s only allow dedicated inputs or outputs, no inout signals
  * Signals passed as output can not be read from (to be discussed, if that should change)
* Auxiliary functions must -- for now -- be translated to a hardware unit (TBD: @hdlfunction inference)


As *hardware generators* in general are considered functions that are decorated using a

* `@always`, `@always_comb`, `@always_seq`

as opposed to simulation constructs not generating hardware in particular: `@instance`

MyHDL hardware generators are translated to the *IRL* -- intermediate representation language (which is just executable Python using a meta programming functional approach).

To use MyHDL code in emulation mode:

In [1]:
from myirl.emulation.myhdl import *

Then we define an auxiliary for cheap unit testing using VHDL-93 dialect and GHDL for reference testing:

In [2]:
def test(uut, debug = False):
    inst = uut()
    vhdl93 = targets.vhdl.VHDL93()
    f = inst.elab(vhdl93, elab_all=True)
    run_ghdl(f, inst, debug = debug, std = '93', vcdfile = inst.name + '.vcd')

## Test unit

The following block is a simple example to start from. Modifications of this base will exhibit a few pitfalls during migration:

In [3]:
@block
def unit():
    a = Signal(intbv(0xaa)[8:])
    a.init = True
    q = Signal(bool())

    @instance
    def stim():
        q.next = False
        if a[0] == True and a[1] == False and a[7] == False:  # True boolean evaluation
            q.next = True
            
        yield delay(1)
        assert q == False
    
    return instances()


test(unit)

Creating sequential 'unit/stim' 
 Insert unit unit 
 DEBUG: Writing 'unit' to file /tmp/unit.vhdl 
Finished _elab in 0.0028 secs
DEBUG Creating library file /tmp/module_defs.vhdl


### Variations

Instead of creating an auxiliary signal, we can create references to signal combinations inside the `@block` context.
However, this may create redundant code when resolving to a HDL, as the reference is a true Python IRL object.

In [4]:
@block
def unit():
    a = Signal(intbv(0xaa)[8:])
    a.init = True
    p, q = [ Signal(bool()) for _ in range(2) ]
    
    z = (a[7] == True) & (a[6] == False) & (a[0] == True) # Reference to binary combination of boolean expressions
    
    zb = a[7] & ~a[6] & a[0]  # True binary combination
    
    zs = Signal(bool())
    
    # New wiring 'generator' construct:
    wires = [
        zs.set(zb)
    ]

    @instance
    def stim():
        q.next = False
        p.next = True
        
        yield delay(1)
        if a[7] == True and a[6] == False and a[0] == True:  # True boolean evaluation
            q.next = True
            
        yield delay(1)
            
        if z:  # Evaluate reference
            q.next = True
            
        yield delay(1)

        if zb: # Evaluate binary op reference
            q.next = True
            
        if zs == True: # Check signal
            q.next = True
            
        yield delay(1)
        assert q == False

        a.next = 0xa1
        yield delay(1)
        p.next = z
        yield delay(1)

        assert p == True
    
    return instances()


test(unit, debug = True)

Creating sequential 'unit/stim' 
 Insert unit unit 
 DEBUG: Writing 'unit' to file /tmp/unit.vhdl 
Finished _elab in 0.0355 secs
DEBUG Creating library file /tmp/module_defs.vhdl
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/unit.vhdl
elaborate unit

==== COSIM stderr ====

==== COSIM stdout ====

==== COSIM stderr ====



In [5]:
!cat -n {unit.ctx.path_prefix}unit.vhdl

     1	-- File generated from /usr/local/lib/python3.8/runpy.py
     2	-- (c) 2016-2021 section5.ch
     3	-- Modifications may be lost
     4	
     5	library IEEE;
     6	use IEEE.std_logic_1164.all;
     7	use IEEE.numeric_std.all;
     8	
     9	library work;
    10	
    11	use work.txt_util.all;
    12	use work.myirl_conversion.all;
    13	
    14	entity unit is
    15	end entity unit;
    16	
    17	architecture MyIRL of unit is
    18	    -- Local type declarations
    19	    -- Signal declarations
    20	    signal q : std_ulogic;
    21	    signal p : std_ulogic;
    22	    signal a : unsigned(7 downto 0) := x"aa";
    23	    signal zs : std_ulogic;
    24	begin
    25	    
    26	stim:
    27	    process
    28	    begin
    29	        q <= '0';
    30	        p <= '1';
    31	        wait for 1 ns;
    32	        if (((a(7) = '1') and (a(6) = '0')) and (a(0) = '1')) then
    33	            q <= '1';
    34	        end if;
    35	        wait for 1 ns;
    36	        if (((a(7

In [6]:
!ls *.vcd

bs.vcd	  tb_gray.vcd	testbench.vcd  unit1.vcd
hyst.vcd  tb_gray1.vcd	unit.vcd       yuv.vcd


## Variable usage

# Issues

* Not translating to binary in `clkgen()`:

In [7]:
@block
def unit():
    @always(delay(1))
    def clkgen():
        clk.next = not clk
        
    return instances()

print(unit.unparse())

Unparsing unit unit


@block
def unit():

    @always_(delay(1))
    def clkgen():
        (yield [clk.set((~ clk))])
    return instances()



/home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/emulation/myhdl2irl.py:515: UserWarning: Replacing logical not by inversion
  warnings.warn("Replacing logical not by inversion")


In [8]:
@block
def unit(a : Signal, b: Signal.Output, * , PARAM):
    @always_comb
    def worker():
        if a == 5:
            b.next = 0
        elif PARAM:
            b.next = 1
    return instances()
print(unit.unparse())

@block
def tb():
    a, b = (Signal(intbv()[5:]) for _ in range(2))
    uut = unit(a, b, True)
    
    return instances()

Unparsing unit unit


@block
def unit(a: Signal, b: Signal.Output, *, PARAM):

    @always_comb_
    def worker():
        (yield [If((a == 5)).Then(b.set(0)).Elif(PARAM).Then(b.set(1))])
    return instances()



In [9]:
test(tb, debug = True)

 use default parameter PARAM : <empty> 
 Insert unit unit_s5_s5 
 Insert unit tb 


TypeError: ('/tmp/ipykernel_21316/3382091292.py:worker()', 'Line 7', "Unexpected condition type <class 'myirl.kernel.components.Parameter'>")